# Problem set 3
## Exercise 5

---

&nbsp;

### Imports:

In [6]:
import pickle
import numpy as np
from mountain.envs.mountain import MountainEnv

&nbsp;

### Task 1:

> Load the trajectories and perform MC evaluation for $v_{\pi}(s)$.

In [ ]:
pickle.load(open('mountain.pkl', 'rb'))